In [3]:
import re
from string import punctuation

import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from joblib import delayed, Parallel


def clear_text(text):
    """Function to clean a single text."""
    lemmatizer = WordNetLemmatizer()

    # Преобразование к нижнему регистру до удаления знаков пунктуации
    text = text.lower()

    # Компиляция регулярных выражений
    url_pattern = re.compile(r"https?://[^,\s]+,?")
    mention_pattern = re.compile(r"@[^,\s]+,?")

    # Удаление URL
    text = re.sub(url_pattern, "", text)

    # Удаление упоминаний
    text = re.sub(mention_pattern, "", text)

    # Удаление знаков пунктуации
    text = text.translate(str.maketrans("", "", punctuation))

    # Удаление лишних пробелов
    text = re.sub(" +", " ", text)

    # Преобразование текста в список токенов
    text_tokens = word_tokenize(text)

    # Инициализация множества стоп-слов
    stop_words = set(stopwords.words("english"))

    # Лемматизация слов и исключение стоп-слов
    lemma_text = [
        lemmatizer.lemmatize(word) for word in text_tokens
        if word not in stop_words
    ]

    # Соединение лемматизированных слов в строку
    cleaned_text = " ".join(lemma_text)

    return cleaned_text


def clear_data(source_path: str, target_path: str, n_jobs: int):
    """Parallel process dataframe

    Parameters
    ----------
    source_path : str
        Path to load dataframe from

    target_path : str
        Path to save dataframe to

    n_jobs : int
        Count of job to process
    """
    data = pd.read_parquet(source_path).dropna().reset_index(drop=True)

    cleaned_text_list = Parallel(n_jobs=n_jobs, backend="multiprocessing")(
        delayed(clear_data)(text) for text in data["text"]
    )

    data["cleaned_text"] = cleaned_text_list
    data.to_parquet(target_path)


ModuleNotFoundError: No module named 'nltk'

In [2]:
!pip install nltk